## Python Blackjack
For this project you will make a Blackjack game using Python. Click <a href="http://www.hitorstand.net/strategy.php">here</a> to familiarize yourself with the the rules of the game. You won't be implementing every rule "down to the letter" with the game, but we will doing a simpler version of the game. This assignment will be given to further test your knowledge on object-oriented programming concepts.

### Rules:

`1. ` The game will have two players: the Dealer and the Player. The game will start off with a deck of 52 cards. The 52 cards will consist of 4 different suits: Clubs, Diamonds, Hearts and Spades. For each suit, there will be cards numbered 1 through 13. <br>
**Note: No wildcards will be used in the program**

`2. ` When the game begins, the dealer will shuffle the deck of cards, making them randomized. After the dealer shuffles, it will deal the player 2 cards and will deal itself 2 cards from. The Player should be able to see both of their own cards, but should only be able to see one of the Dealer's cards.
 
`3. ` The objective of the game is for the Player to count their cards after they're dealt. If they're not satisfied with the number, they have the ability to 'Hit'. A hit allows the dealer to deal the Player one additional card. The Player can hit as many times as they'd like as long as they don't 'Bust'. A bust is when the Player is dealt cards that total more than 21.

`4. ` If the dealer deals the Player cards equal to 21 on the **first** deal, the Player wins. This is referred to as Blackjack. Blackjack is **NOT** the same as getting cards that equal up to 21 after the first deal. Blackjack can only be attained on the first deal.

`5. ` The Player will never see the Dealer's hand until the Player chooses to 'stand'. A Stand is when the player tells the dealer to not deal it anymore cards. Once the player chooses to Stand, the Player and the Dealer will compare their hands. Whoever has the higher number wins. Keep in mind that the Dealer can also bust. 

In [138]:
# Contains all the requirements for a game
class Game():
    def __init__(self):
        self.deck = Deck().deck
        self.dealer = Dealer()
        self.player = Player()
    
        
class Person():
    def __init__(self, person_type):
        self.cards = []
        self.person_type = person_type.lower()
        self.hand_total = 0
    
    # Calculates the sum of player/dealer's hand
    def calculate_hand(self):
        self.hand_total = sum([value for value, suit in self.cards])
    
    # Prints player and dealers cards
    def show_cards(self, end_of_game=False):
        # Formats the print string with the first card
        print_cards = f'{self.person_type.title()} has {self.cards[0][0]} of {self.cards[0][1]}'
        for i in range(1, len(self.cards)):
            print_cards += ' -'
            # prints all the player cards or the deaelers card when the game is over
            if self.person_type == 'player' or (self.person_type == 'dealer' and end_of_game):
                # Prints Ace and Royals instead of numeric values
                if self.cards[i][0] == 1:
                    print_cards += f'Ace of {self.cards[i][1]} '
                elif self.cards[i][0] == 11:
                    print_cards += f'Jack of {self.cards[i][1]} '
                elif self.cards[i][0] == 12:
                    print_cards += f'Queen of {self.cards[i][1]} '
                elif self.cards[i][0] == 13:
                    print_cards += f'King of {self.cards[i][1]} '  
                else:
                    print_cards += f'{self.cards[i][0]} of {self.cards[i][1]} '
            else:
                print_cards += 'HIDDEN '
        
        print(print_cards)
            
class Dealer(Person):
    def __init__(self):
        super().__init__('dealer')
            
class Player(Person):
    def __init__(self, balance=100):
        super().__init__('player')
        self.balance = balance
    
    # Prints the current player balance
    def print_balance(self):
        print('Your current balance is', self.balance)
    
    # Prompts user to enter a bet
    def bet(self):
        self.print_balance()
        return int(input('Please enter your bet: '))
    
    # Calculates new player balance depending on winning status
    def calculate_balance(self, status, bet_amount):
        if status.lower() == 'win':
            self.balance += bet_amount
        elif status.lower() == 'lose':
            self.balance -= bet_amount
        
        self.print_balance()
            

class Deck():
    from random import shuffle
    
    # Generates a deck when a new instance is created
    def __init__(self):
        self.deck = self.generate_deck()
        self.shuffle_deck()
    
    # Generates a new deck of 52 cards
    def generate_deck(self):
        suits = ['Diamonds', 'Hearts', 'Spades', 'Clubs']
        values = [i for i in range(1, 14)]
        return [(v, s) for v in values for s in suits]
    
    # Shuffles the instance deck
    def shuffle_deck(self):
        shuffle(self.deck)

In [139]:
game = Game()

In [144]:
game.deal_cards('dealer')
game.dealer.show_cards(True)

Dealer has 7 of Spades - 4 of Hearts  - Ace of Diamonds  - 4 of Spades  - 10 of Clubs 


In [145]:
game.dealer.calculate_hand()
game.dealer.hand_total

26

TODO: Need to calculate win conditions (also need to check if the hand has an ace)